In [ ]:
import sisl
import Hubbard.hamiltonian as hh
import Hubbard.sp2 as sp2
import Hubbard.density as density
from Hubbard.negf import NEGF
import Hubbard.plot as plot
%matplotlib inline

# Simulation of electron correlations in open-quantum systems 

In this example we will study the electron correlations in an open-quantum system, and find the self-consistent solution using the non-equilibrium Green's function (NEGF) formalism within the `Hubbard` package.

Here we will reproduce the results presented in [Phys. Rev. B 81, 245402 (2010)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.245402) for the system shown in Fig. 3(a). This is, a zigzag graphene nanoribbon with a defect created by removing certain atoms in the central region. This open-quantum system is composed by a central region that contains the defect coupled to two semi-infinite leads or electrodes (left and right).

For these systems and using the NEGF formalism, we are able to find the transmission probabilities per spin channel.
Look into example [TB_04](https://github.com/zerothi/ts-tbt-sisl-tutorial/blob/master/TB_04/run.ipynb) for more information about how [TBtrans](https://launchpad.net/siesta) and [sisl](https://sisl.readthedocs.io/en/v0.10.0/api.html) can be combined.

We will focus on the equilibrium situation, therefore the temperature and chemical potentials of the electrodes *must coincide*. The complex contour that we use to integrate the density matrix in the `Hubbard.NEGF` class is extracted from a [Transiesta](https://launchpad.net/siesta) calculation performed for a temperature of `kT=0.025` eV, which we will set as common for all the composing element calculations.


In [ ]:
# Set U and kT for the whole calculation (use these values when needed)
U = 2.0
kT = 0.025

#### 1) Compute the electrodes 
We will start by building the tight-binding (TB) Hamiltonian for the graphene nanoribbons, which compose the electrodes (periodic boundary conditions). Then we will find their self-consistent solution by using the `Hubbard` package.

In [ ]:
# Build sisl.Geometry object of a ZGNR of width W=5 C-atoms across,
# e.g., by using the function sisl.geom.zgnr.
# This function returns the unit-cell of a periodic ZGNR along the x-axis.
ZGNR = sisl.geom.zgnr(5)

# Build tight-binding Hamiltonian using sp2 function
H_elec = sp2(ZGNR, t1=2.7, t2=0.2, t3=0.18, s1=0, s2=0, s3=0)

# Build the HubbardHamiltonian object for the periodic system:
MFH_elec = hh.HubbardHamiltonian(..., U=U, kT=kT)

# Initialize electrodes spin density
MFH_elec.set_polarization([0], dn=[9])

# Converge Electrode Hamiltonians
dn = MFH_elec.converge(...)

- Let's plot the spin polarization of the unit cel to check out the self-consistent solution

In [ ]:
# Now we have the converged electrodes saved in the MFH_elec variable
p = plot.SpinPolarization(MFH_elec, colorbar=True, vmax=0.4, vmin=-0.4)

#### 2) Build the central region
The next step is to build the geometry of the whole device. The system is composed by a series of repetitions of the electrode and the defect in the center. We also have to make sure that the device is finite (no periodic boundary conditions).

In [ ]:
# Build central region TB Hamiltonian
HC = H_elec.tile(16, axis=0)

# Remove atoms in the central area to recreate the system of Fig. 3 (a) of the reference paper.
HC = HC.remove([69, 79, 89, 78, 88, 66, 77, 76, 87, 86, 65, 74, 75, 84, 85])

# Make it finite
HC.set_nsc([1, 1, 1])

# Write geometry to file if one wants to visualize it
HC.geometry.write('device.xyz')